<a href="https://colab.research.google.com/github/KBVKarthik/Keras-Computer-Vision-Models/blob/main/Gradient_Centralization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
from time import time

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

In [56]:
num_classes = 2
input_shape = (300, 300, 3)
dataset_name = "horses_or_humans"
batch_size = 128
AUTOTUNE = tf.data.experimental.AUTOTUNE

(train_ds, test_ds), metadata = tfds.load(
    name=dataset_name,
    split=[tfds.Split.TRAIN, tfds.Split.TEST],
    with_info=True,
    as_supervised=True,
)

print(f"Image shape: {metadata.features['image'].shape}")
print(f"Training images: {metadata.splits['train'].num_examples}")
print(f"Test images: {metadata.splits['test'].num_examples}")

Shuffling and writing examples to /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteOYOGVI/horses_or_humans-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteOYOGVI/horses_or_humans-test.tfrecord


Dataset horses_or_humans downloaded and prepared to /root/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.
Image shape: (300, 300, 3)
Training images: 1027
Test images: 256


In [57]:
rescale = layers.experimental.preprocessing.Rescaling(1.0 / 255)

data_augmentation = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        layers.experimental.preprocessing.RandomRotation(0.3),
        layers.experimental.preprocessing.RandomZoom(0.2),
    ]
)


def prepare(ds, shuffle=False, augment=False):

    ds = ds.map(lambda x, y: (rescale(x), y), num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1024)


    ds = ds.batch(batch_size)


    if augment:
        ds = ds.map(
            lambda x, y: (data_augmentation(x, training=True), y),
            num_parallel_calls=AUTOTUNE,
        )


    return ds.prefetch(buffer_size=AUTOTUNE)

In [58]:
train_ds = prepare(train_ds, shuffle=True, augment=True)
test_ds = prepare(test_ds)

In [59]:
model = tf.keras.Sequential(
    [
        layers.Conv2D(16, (3, 3), activation="relu", input_shape=(300, 300, 3)),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(32, (3, 3), activation="relu"),
        layers.Dropout(0.5),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.Dropout(0.5),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(512, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

In [60]:
class GCRMSprop(RMSprop):
    def get_gradients(self, loss, params):


        grads = []
        gradients = super().get_gradients()
        for grad in gradients:
            grad_len = len(grad.shape)
            if grad_len > 1:
                axis = list(range(grad_len - 1))
                grad -= tf.reduce_mean(grad, axis=axis, keep_dims=True)
            grads.append(grad)

        return grads


optimizer = GCRMSprop(learning_rate=1e-4)

In [61]:
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time() - self.epoch_time_start)

In [62]:
time_callback_no_gc = TimeHistory()
model.compile(
    loss="binary_crossentropy",
    optimizer=RMSprop(learning_rate=1e-4),
    metrics=["accuracy"],
)

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_69 (Conv2D)           (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 147, 147, 32)      4640      
_________________________________________________________________
dropout_5 (Dropout)          (None, 147, 147, 32)      0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 71, 71, 64)        18496     
_________________________________________________________________
dropout_6 (Dropout)          (None, 71, 71, 64)       

In [63]:
history_no_gc = model.fit(
    train_ds, epochs=10, verbose=1, callbacks=[time_callback_no_gc]
)

Epoch 1/10
9/9 [==============================] - 23s 1s/step - loss: 0.7099 - accuracy: 0.5599
Epoch 2/10
9/9 [==============================] - 21s 2s/step - loss: 0.6746 - accuracy: 0.5940
Epoch 3/10
9/9 [==============================] - 20s 2s/step - loss: 0.6569 - accuracy: 0.6495
Epoch 4/10
9/9 [==============================] - 21s 2s/step - loss: 0.6367 - accuracy: 0.6573
Epoch 5/10
9/9 [==============================] - 22s 2s/step - loss: 0.5985 - accuracy: 0.6972
Epoch 6/10
9/9 [==============================] - 19s 1s/step - loss: 0.5706 - accuracy: 0.7546
Epoch 7/10
9/9 [==============================] - 22s 2s/step - loss: 0.5548 - accuracy: 0.7468
Epoch 8/10
9/9 [==============================] - 21s 2s/step - loss: 0.4952 - accuracy: 0.7868
Epoch 9/10
9/9 [==============================] - 20s 2s/step - loss: 0.5143 - accuracy: 0.7644
Epoch 10/10
9/9 [==============================] - 21s 2s/step - loss: 0.4651 - accuracy: 0.7907


In [64]:
time_callback_gc = TimeHistory()
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

model.summary()

history_gc = model.fit(train_ds, epochs=10, verbose=1, callbacks=[time_callback_gc])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_69 (Conv2D)           (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 147, 147, 32)      4640      
_________________________________________________________________
dropout_5 (Dropout)          (None, 147, 147, 32)      0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 71, 71, 64)        18496     
_________________________________________________________________
dropout_6 (Dropout)          (None, 71, 71, 64)       

In [65]:
print("Not using Gradient Centralization")
print(f"Loss: {history_no_gc.history['loss'][-1]}")
print(f"Accuracy: {history_no_gc.history['accuracy'][-1]}")
print(f"Training Time: {sum(time_callback_no_gc.times)}")

print("Using Gradient Centralization")
print(f"Loss: {history_gc.history['loss'][-1]}")
print(f"Accuracy: {history_gc.history['accuracy'][-1]}")
print(f"Training Time: {sum(time_callback_gc.times)}")

Not using Gradient Centralization
Loss: 0.46512818336486816
Accuracy: 0.7906523942947388
Training Time: 209.66993641853333
Using Gradient Centralization
Loss: 0.34362277388572693
Accuracy: 0.8558909296989441
Training Time: 210.90795469284058
